<a href="https://colab.research.google.com/github/Harshbhatt2409/Text-Matching-System-for-Question-Retrieval/blob/main/a1872953_ANLP_Assignement_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**a1872953**

**Harsh Alpesh Bhatt**

**Applied Natural Language Processing Assignment 2**

**Task: Building a text matching system for question matching**

In [ ]:
# Importing the required libraries

import pandas as pd
import re as re
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter

##1. Importing Dataset and doing text preprocessing

In [ ]:
df = pd.read_csv('/content/data.tsv', sep='\t', error_bad_lines=False)
df

<ipython-input-2-6489edb9910c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/data.tsv', sep='\t', error_bad_lines=False)
Skipping line 83032: expected 6 fields, saw 7

Skipping line 154657: expected 6 fields, saw 7

Skipping line 323916: expected 6 fields, saw 7

<ipython-input-2-6489edb9910c>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data.tsv', sep='\t', error_bad_lines=False)


,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


Now we will create two copies of the original dataset. One copy will be used for TF-IDF part and one for GloVe part

In [ ]:
df1 = df.copy()
df2 = df.copy()

Copy one of the original dataset

In [ ]:
df1

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


Copy Two of the original Dataset

In [ ]:
df2

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


Finding and dropping null values from the dataset

In [ ]:
#checking if the dataset has any null values
print(df1.isnull().sum())

id               0
qid1             0
qid2             7
question1       11
question2       12
is_duplicate    12
dtype: int64


In [ ]:
#dropping the null values from the dataset
df1.dropna(inplace = True)
print(df1.isnull().sum())

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [ ]:
df1.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [ ]:
df1.shape

(363177, 6)

In [ ]:
df1.info()
df1.describe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


<bound method NDFrame.describe of             id    qid1      qid2  \
0       133273  213221  213222.0   
1       402555  536040  536041.0   
2       360472  364011  490273.0   
3       150662  155721    7256.0   
4       183004  279958  279959.0   
...        ...     ...       ...   
363187   57443  100941   83372.0   
363188  278260   62873   34460.0   
363189  136211  217377  217378.0   
363190  302720  425744  285638.0   
363191  126322   39774   20105.0   

                                                question1  \
0       How is the life of a math student? Could you d...   
1                     How do I control my horny emotions?   
2            What causes stool color to change to yellow?   
3                             What can one do after MBBS?   
4       Where can I find a power outlet for my laptop ...   
...                                                   ...   
363187               How do I make money flying my drone?   
363188          What can you do with an econo

In [ ]:
df1.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

Now we know that this dataset has 6 columns id, qid1, qid2, question1, question2, is_duplicate

In [ ]:
#lets print some of the questions, 1 from each column
df1['question1'][0]

'How is the life of a math student? Could you describe your own experiences?'

In [ ]:
df1['question2'][0]

'Which level of prepration is enough for the exam jlpt5?'

We can see that text preprocessing is required for the dataset

In [ ]:
# defining function to convert to lower case
def lower(data):
  return data.lower()

Now lets apply this to question1

In [ ]:
#for column question 1
df1['question1'] = df1['question1'].apply(lower)

Checking whether the function has worked for question2

In [ ]:
#checking
df1['question1'][0]

'how is the life of a math student? could you describe your own experiences?'

Now we will apply for Question2

In [ ]:
#for column question 2
df1['question2'] = df1['question2'].apply(lower)

Checking for Question2

In [ ]:
df1['question2'][0]

'which level of prepration is enough for the exam jlpt5?'

After lower casing we will remove html tags and special characters

In [ ]:
# defining function to remove HTML tags
def html_tags(data):
  clean = re.compile("<.*?>")
  return re.sub(clean, '', data)

applying function on question1

In [ ]:
#for column question 1
df1['question1'] = df1['question1'].apply(html_tags)

appying function for question2

checking for question1

In [ ]:
#checking
df1['question1'][0]

'how is the life of a math student? could you describe your own experiences?'

checking for question2

In [ ]:
#for column question 2
df1['question2'] = df1['question2'].apply(html_tags)

function to remove special characters

In [ ]:
#function to remove special characters
def special_character_removal(data):
  g = ''
  for t in data:
    if t.isalnum():
      g = g + t
    else:
      g = g + ' '
  return g

Applying this to question1

In [ ]:
#for column question 1
df1['question1'] = df1['question1'].apply(special_character_removal)

Checking for question1

In [ ]:
#checking
df1['question1'][0]

'how is the life of a math student  could you describe your own experiences '

Applying to question2

In [ ]:
#for column question 2
df1['question2'] = df1['question2'].apply(special_character_removal)

Checking for question2

In [ ]:
df1['question2'][0]

'which level of prepration is enough for the exam jlpt5 '

Now lets remove all the stop words

In [ ]:
#function to remove stop words
#we will use the nltk library for this process
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#specifing the predefined stopwords library to find stopwords in english language
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Now we will remove all the stop words from the datasets

First we will do for question1 of df1

In [ ]:
df1['question1'] = df1['question1'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

Checking for question1

In [ ]:
#checking
df1['question1'][0]

'life math student could describe experiences'

Applying for question2

In [ ]:
df1['question2'] = df1['question2'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

Once question1 is completed we will also repeat the step for question2

Checking for question2

In [ ]:
df1['question2'][0]

'level prepration enough exam jlpt5'

Now we will perfrom the Tokenization and Lemmatization of the data

In [ ]:
#creating a function for tokenization
def tokenization(text):
  tokens = nltk.wordpunct_tokenize(text)
  return tokens

Applying tokenization to question1

In [ ]:
df1['question1'] = df1['question1'].apply(tokenization)

Checking the results after tokenization

In [ ]:
#checking
df1['question1'][0]

['life', 'math', 'student', 'could', 'describe', 'experiences']

Applying tokenization for question2

In [ ]:
df1['question2'] = df1['question2'].apply(tokenization)

Checking for question2

In [ ]:
df1['question2'][0]

['level', 'prepration', 'enough', 'exam', 'jlpt5']

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wnetlemma = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


We will now create a Lemmatisation function to perform the step of lemmatization

In [ ]:
#creating a function for lemmatization
def lemmatisation(text):
  words_lemma = [wnetlemma.lemmatize(w) for w in text]
  return words_lemma

we will now apply this function that we have just created to the df1, starting with column question1

we will repeat the process for question2

In [ ]:
df1['question1'] = df1['question1'].apply(lemmatisation)

Applying Lemmatization for question2

In [ ]:
df1['question2'] = df1['question2'].apply(lemmatisation)

Checking for question1

In [ ]:
#checking
df1['question1'][0]

['life', 'math', 'student', 'could', 'describe', 'experience']

Checking for question2

In [ ]:
df1['question2'][0]

['level', 'prepration', 'enough', 'exam', 'jlpt5']

Now we will perfrom reset_index operation to reset the index back to as they were


In [ ]:
df1.reset_index(inplace=True)

Taking only the duplicate functions

In [ ]:
#now we will get the queries with the duplicate column as 1.0
queries = df1[df1['is_duplicate']!=0]
queries

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
7,7,106969,147570,787.0,"[best, self, help, book, read, change, life]","[top, self, help, book, read]",1.0
11,11,233239,71243,177376.0,"[hillary, clinton, policy, towards, india, bec...","[hilary, clinton, policy, towards, india, beco...",1.0
13,13,11568,22332,22333.0,"[best, book, study, tensor, general, relativit...","[best, book, tensor, calculus]",1.0
...,...,...,...,...,...,...,...
363168,363183,300337,423121,423122.0,"[get, best, life]","[get, best, life]",1.0
363170,363185,344766,473025,473026.0,"[demand, oracle, atg, market]","[demand, oracle, atg, market]",1.0
363172,363187,57443,100941,83372.0,"[make, money, flying, drone]","[use, dji, phantom, make, money]",1.0
363173,363188,278260,62873,34460.0,"[economics, degree]","[job, get, economics, degree]",1.0


Now these has 134141 rows so to change this we will only take the top 100 and name them queries_100

In [ ]:
queries_100 = queries[:100]

In [ ]:
queries_100.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
7,7,106969,147570,787.0,"[best, self, help, book, read, change, life]","[top, self, help, book, read]",1.0
11,11,233239,71243,177376.0,"[hillary, clinton, policy, towards, india, bec...","[hilary, clinton, policy, towards, india, beco...",1.0
13,13,11568,22332,22333.0,"[best, book, study, tensor, general, relativit...","[best, book, tensor, calculus]",1.0


In [ ]:
queries_100.shape

(100, 7)

Now for our convinence lets rearrange the columns so that qid1 and question1 are next to each other and same for qid2 and question2, we will also reset the index once the columns are rearranged.

In [ ]:
column = ["qid1","question1","qid2","question2","is_duplicate"]
queries_100 = queries_100[column]


Reseting Index and checking

In [ ]:
queries_100 = queries_100.reset_index(drop=True)
queries_100.head()

,qid1,question1,qid2,question2,is_duplicate
0,536040,"[control, horny, emotion]",536041.0,"[control, horniness]",1.0
1,155721,"[one, mbbs]",7256.0,[mbbs],1.0
2,147570,"[best, self, help, book, read, change, life]",787.0,"[top, self, help, book, read]",1.0
3,71243,"[hillary, clinton, policy, towards, india, bec...",177376.0,"[hilary, clinton, policy, towards, india, beco...",1.0
4,22332,"[best, book, study, tensor, general, relativit...",22333.0,"[best, book, tensor, calculus]",1.0


After text preprocessing is complete we move onto TF-IDF

##2. TF-IDF and Matching Steps

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.

This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

In [ ]:
#creating vocabulary
#for the first step here we will create the word ids and get the vocab in token form
#we will take question2 as the list
df_question = df1['question2']
id_of_words = {}
token_vocab = df_question.tolist()
word_vocab = None
temporary_vocab = []
unique_word = {}
list_of_vocabulary = []
for word in token_vocab:
  for val in word:
    temporary_vocab.append(val)
length_of_vocabulary = len(list_of_vocabulary)
token_vocab = temporary_vocab
data_set = set(token_vocab)
word_vocab
token_vocab = list(data_set)
length_total_vocab = len(token_vocab)

All the vocab in form as tokens

In [ ]:
token_vocab

['ensuring',
 'abnormality',
 'mangaloreans',
 'olufsen',
 'recreated',
 'a7000',
 'ripping',
 'erode',
 'unwittingly',
 'skydiving',
 'perminant',
 'accrual',
 'protous',
 'robespierre',
 'opencv',
 'filmmaker',
 'eligibility',
 'cpct',
 '7500u',
 '1956',
 'hougyoku',
 'ordering',
 'nôm',
 'photogrammetry',
 'presiding',
 'refined',
 'fiddler',
 'bulma',
 'harzard',
 'unstressed',
 'hopstop',
 'honduras',
 '4pm',
 'keratin',
 'muskie',
 'hippie',
 'sequential',
 'exterminator',
 'meep',
 'allergen',
 'auroville',
 'hovers',
 'jio4g',
 'van',
 'waving',
 'glycerine',
 'lot',
 'colonized',
 'separately',
 'sportsdear',
 'oswego',
 'advantges',
 'snapbacks',
 'haunted',
 'lokenath',
 '20yrs',
 'uwaterloo',
 'gestapo',
 'dysp',
 'velcro',
 'resoning',
 'sibm',
 'kunal',
 'peyton',
 'cheeseburger',
 'girfriend',
 'iyer',
 'repeater',
 'nutrex',
 'cystic',
 'smbs',
 'circ4',
 'evan',
 'solo',
 'maccallum',
 'database',
 'flightaware',
 '155mbs',
 'torrance',
 'pushover',
 'ramasamy',
 'laer

Now the length of the vacbulary generated above is as follows

In [ ]:
#length of the vocabulary
length_total_vocab

54318

We will now write a function to generate vocabulary in the form of a list

In [ ]:
#generating the vocab in the form of a list
#defining a func to do the same
def vocab_as_list():
  for data in token_vocab:
    id_of_words[data] = list()
def dictionary_vocabulary():
  for item in range(len(df1)):
    return id_of_words[data]
#now for question 2 and qid2
def df_question_2():
  for item in range(len(df1)):
    df_question2 = []
    for data in df1.loc[item, 'question2']:
      qid2_of_data = df1.loc[item, 'qid2']
      ultimate_data = qid2_of_data
      id_of_words[data][len(id_of_words[data]):] = [ultimate_data]
def vocabulary_of_tokens():
  for data in token_vocab:
    return id_of_words[data]
def token_vocabulary():
  for data in token_vocab:
    data_cleaned = set(id_of_words[data])
    id_of_words[data] = list(data_cleaned)
vocab_as_list()
df_question_2()
token_vocabulary()
id_of_items = list(id_of_words.items())

Now lets check the above functions by printing the ids of the 20 items

In [ ]:
print(id_of_items[:1])

[('ensuring', [148696.0, 220880.0])]


In [ ]:
print("ID from the list with ids from 1-20", id_of_items[:20])

ID from the list with ids from 1-20 [('ensuring', [148696.0, 220880.0]), ('abnormality', [374632.0, 388242.0]), ('mangaloreans', [160962.0]), ('olufsen', [229313.0]), ('recreated', [394400.0, 470519.0]), ('a7000', [187554.0, 424963.0, 503398.0, 502856.0, 476523.0, 371536.0]), ('ripping', [109976.0, 333612.0, 521919.0]), ('erode', [245384.0, 346468.0]), ('unwittingly', [242243.0, 465998.0]), ('skydiving', [29024.0, 207521.0, 263622.0, 225991.0, 73802.0, 431728.0, 350545.0, 320182.0, 82134.0, 222326.0]), ('perminant', [394800.0]), ('accrual', [319540.0, 227415.0]), ('protous', [355458.0]), ('robespierre', [36816.0]), ('opencv', [220481.0, 428705.0, 49220.0, 268875.0, 189068.0, 179219.0, 165075.0, 349080.0, 349079.0, 472792.0, 502429.0]), ('filmmaker', [292292.0, 136199.0, 522890.0, 153356.0, 234925.0, 447503.0, 381615.0, 57711.0, 434833.0, 274799.0, 477684.0, 447504.0, 404913.0, 211255.0, 455226.0, 506364.0, 329534.0]), ('eligibility', [33664.0, 492419.0, 73348.0, 146437.0, 216329.0, 601

We will now write a function to calculate the Inverse of the above generated list

In [ ]:
#Tf-IDF
Inverse_list = {}
x = df1['qid1']
unique_x = x.nunique()
inverse_value = len(x)
tmp_vcb_value =[]
for data in id_of_words.keys():
  lenght_of_dataword = (len(id_of_words[data]) + 1)
  Number = (1 + unique_x / lenght_of_dataword)
  Number_1 = Number+1
  Inverse_list[data] = np.log(Number_1)
Item_Inverse = list(Inverse_list.items())

Printing the inverse list we just made

In [ ]:
Inverse_list

{'ensuring': 11.49219540774383,
 'abnormality': 11.49219540774383,
 'mangaloreans': 11.897653709519384,
 'olufsen': 11.897653709519384,
 'recreated': 11.49219540774383,
 'a7000': 10.644924772223822,
 'ripping': 11.204520141578337,
 'erode': 11.49219540774383,
 'unwittingly': 11.49219540774383,
 'skydiving': 10.192966872606789,
 'perminant': 11.897653709519384,
 'accrual': 11.49219540774383,
 'protous': 11.897653709519384,
 'robespierre': 11.897653709519384,
 'opencv': 10.10596230153286,
 'filmmaker': 9.700538027946205,
 'eligibility': 8.513651333214879,
 'cpct': 11.897653709519384,
 '7500u': 11.897653709519384,
 '1956': 10.981383396504087,
 'hougyoku': 11.897653709519384,
 'ordering': 9.757689636148363,
 'nôm': 11.897653709519384,
 'photogrammetry': 11.897653709519384,
 'presiding': 11.897653709519384,
 'refined': 10.644924772223822,
 'fiddler': 11.897653709519384,
 'bulma': 11.897653709519384,
 'harzard': 11.897653709519384,
 'unstressed': 11.49219540774383,
 'hopstop': 11.89765370951

In [ ]:
#zipping the questions with its relevant ids
zipping_df = zip(df1['qid2'], df1['question2'])
data_word = dict(zipping_df)
item_document = data_word.items()

In [ ]:
data_word

{213222.0: ['level', 'prepration', 'enough', 'exam', 'jlpt5'],
 536041.0: ['control', 'horniness'],
 490273.0: ['cause', 'stool', 'come', 'little', 'ball'],
 7256.0: ['mbbs'],
 279959.0: ['would',
  'second',
  'airport',
  'sydney',
  'australia',
  'needed',
  'high',
  'speed',
  'rail',
  'link',
  'created',
  'melbourne',
  'sydney'],
 193388.0: ['beleive',
  'bulimic',
  'force',
  'throw',
  'atleast',
  'day',
  'eat',
  'something',
  'feel',
  'guilty',
  'tell',
  'somebody'],
 96457.0: ['become', 'air', 'traffic', 'controller'],
 787.0: ['top', 'self', 'help', 'book', 'read'],
 297540.0: ['university',
  'philippine',
  'take',
  'second',
  'bfa',
  'college',
  'fine',
  'art',
  'exempted',
  'gen',
  'ed',
  'core',
  'subject'],
 371478.0: ['move', 'jamaica'],
 4854.0: ['county',
  'edgware',
  'lifestyle',
  'compare',
  'london',
  'borough',
  'enfield'],
 177376.0: ['hilary',
  'clinton',
  'policy',
  'towards',
  'india',
  'become',
  'president'],
 2085.0: ['q

Now we will print the values to check

In [ ]:
print(list(item_document)[:10])

[(213222.0, ['level', 'prepration', 'enough', 'exam', 'jlpt5']), (536041.0, ['control', 'horniness']), (490273.0, ['cause', 'stool', 'come', 'little', 'ball']), (7256.0, ['mbbs']), (279959.0, ['would', 'second', 'airport', 'sydney', 'australia', 'needed', 'high', 'speed', 'rail', 'link', 'created', 'melbourne', 'sydney']), (193388.0, ['beleive', 'bulimic', 'force', 'throw', 'atleast', 'day', 'eat', 'something', 'feel', 'guilty', 'tell', 'somebody']), (96457.0, ['become', 'air', 'traffic', 'controller']), (787.0, ['top', 'self', 'help', 'book', 'read']), (297540.0, ['university', 'philippine', 'take', 'second', 'bfa', 'college', 'fine', 'art', 'exempted', 'gen', 'ed', 'core', 'subject']), (371478.0, ['move', 'jamaica'])]


In [ ]:
def normalize(ls_word, norm = 'l2'):
  word_vector = []
  if norm == 'l1':
    return word_vector[0]
  elif norm == 'l2':
    return word_vector[1]
  else:
    pass

Creating a normalization function to separate the two normalization techniques
L1 Normalizationa and L2 Normalization

In [ ]:
#Normalizing the edited data
def normalization(ls_word, norm ='l2'):
  vector_of_word = []
  for wordd in set(ls_word):
    vector_of_word.append(ls_word.count(wordd))
  vector_word = []
  vector_words = np.array(vector_of_word)
  word_of_vectors = {}
  if norm == 'l1':
    h = np.linalg.norm(vector_words, ord=1)
  elif norm == 'l2':
    h = np.linalg.norm(vector_words, ord=2)
  else:
    pass
  return h

Now we will create an inverse of the values we found and then multiply the two metrics

In [ ]:
#TF inverse for norm L1
TF_Inverse_L1 = dict()
#TF-IDF inverse for norm L1
TF_IDF_L1 = dict()
#inverse of data
invert_data = Inverse_list[data]
def dictionary_vocab_token():
  for data in token_vocab:
    tfdata = dict()
    vocabs = []
    documents ={}
    for doc in id_of_words[data]:
      h_word = data_word[doc]
      number = h_word.count(data)
      vocabs
      denote = normalization(h_word, 'l1')
      tfdata[doc] = number / denote
      documents = vocabs / denote
    TF_Inverse_L1[data] = tfdata

dictionary_vocab_token()

In [ ]:
def token_vocab_list():
  for data in token_vocab:
    tfidf = list()
    vocabs = []
    token_documents = []
    list_of_vocab = []
    item_tf = TF_Inverse_L1[data].items()
    for tp in list(item_tf):
      tp = list(tp)
      tf_idf = tp[1] * invert_data
      vocabs
      tp[1] = tf_idf
      tfidf[len(tfidf):] = [tp]

    TF_IDF_L1[data] = tfidf
token_vocab_list()

Now we will loop this function so that all the values fill the list TF_IDF_L1

In [ ]:
for item in range(len(TF_IDF_L1)):
  tmp = token_vocab[item]
  TF_IDF_L1[tmp] = {v[0]: v[1] for v in TF_IDF_L1[tmp]}

Taking the top 5 elements

In [ ]:
har = list(TF_IDF_L1.items())
har[:5]

[('ensuring', {148696.0: 1.8674200235963894, 220880.0: 0.8003228672555954}),
 ('abnormality', {374632.0: 2.8011300353945843, 388242.0: 2.8011300353945843}),
 ('mangaloreans', {160962.0: 2.8011300353945843}),
 ('olufsen', {229313.0: 2.2409040283156676}),
 ('recreated', {394400.0: 0.9337100117981947, 470519.0: 1.8674200235963894})]

In [ ]:
TF_IDF_L1

{'ensuring': {148696.0: 1.8674200235963894, 220880.0: 0.8003228672555954},
 'abnormality': {374632.0: 2.8011300353945843, 388242.0: 2.8011300353945843},
 'mangaloreans': {160962.0: 2.8011300353945843},
 'olufsen': {229313.0: 2.2409040283156676},
 'recreated': {394400.0: 0.9337100117981947, 470519.0: 1.8674200235963894},
 'a7000': {187554.0: 2.2409040283156676,
  424963.0: 1.4005650176972921,
  503398.0: 0.7002825088486461,
  502856.0: 2.8011300353945843,
  476523.0: 1.6006457345111909,
  371536.0: 2.2409040283156676},
 'ripping': {109976.0: 2.8011300353945843,
  333612.0: 1.4005650176972921,
  521919.0: 1.4005650176972921},
 'erode': {245384.0: 1.4005650176972921, 346468.0: 0.7002825088486461},
 'unwittingly': {242243.0: 0.8618861647367952, 465998.0: 1.018592740143485},
 'skydiving': {29024.0: 3.7348400471927787,
  207521.0: 2.8011300353945843,
  263622.0: 1.4005650176972921,
  225991.0: 2.8011300353945843,
  73802.0: 5.6022600707891685,
  431728.0: 3.2012914690223817,
  350545.0: 5.60

Similarly we will do the same for norm L2

Creating the inverse of the function with normalization L2

In [ ]:
#Tf inverse for Norm L2
TF_Inverse_L2 = dict()
#TF IDF of Norm L2
TF_IDF_L2 = dict()
invert_data = Inverse_list[data]

def dictionary_vocab_token_1():
  for data in token_vocab:
    tfdata = dict()
    vocabs = []
    documents1 = []
    for doc in id_of_words[data]:
      b_word = data_word[doc]
      number = b_word.count(data)
      vocabs
      denote = normalization(b_word, 'l2')
      tfdata[doc] = number / denote
      documents1 = documents1
    TF_Inverse_L2[data] = tfdata

dictionary_vocab_token_1()

In [ ]:
def token_vocab_list_1():
  for data in token_vocab:
    tfidf2 = list()
    vocabs = []
    list_of_vocabs_1 = []
    item_tf2 = TF_Inverse_L2[data].items()
    for tp in list(item_tf2):
      tp = list(tp)
      tf_idf = tp[1] * invert_data
      vocabs
      tp[1] = tf_idf
      tfidf2[len(tfidf2):] = [tp]

    TF_IDF_L2[data] = tfidf2
token_vocab_list_1()

Now we will loop this function so that all the values fill the list TF_IDF_L2

In [ ]:
for item in range(len(TF_IDF_L2)):
  tmp = token_vocab[item]
  TF_IDF_L2[tmp] = {v[0]: v[1] for v in TF_IDF_L2[tmp]}

Taking the top 5 elements

In [ ]:
hars = list(TF_IDF_L2.items())
hars[:5]

[('ensuring', {148696.0: 4.574226193267277, 220880.0: 2.8011300353945843}),
 ('abnormality', {374632.0: 5.6022600707891685, 388242.0: 5.6022600707891685}),
 ('mangaloreans', {160962.0: 5.6022600707891685}),
 ('olufsen', {229313.0: 5.010813738366946}),
 ('recreated', {394400.0: 3.2344663599404186, 470519.0: 4.574226193267277})]

In [ ]:
TF_IDF_L2

{'ensuring': {148696.0: 4.574226193267277, 220880.0: 2.8011300353945843},
 'abnormality': {374632.0: 5.6022600707891685, 388242.0: 5.6022600707891685},
 'mangaloreans': {160962.0: 5.6022600707891685},
 'olufsen': {229313.0: 5.010813738366946},
 'recreated': {394400.0: 3.2344663599404186, 470519.0: 4.574226193267277},
 'a7000': {187554.0: 5.010813738366946,
  424963.0: 3.5431803736619822,
  503398.0: 2.8011300353945843,
  502856.0: 5.6022600707891685,
  476523.0: 4.234910550632928,
  371536.0: 5.010813738366946},
 'ripping': {109976.0: 5.6022600707891685,
  333612.0: 3.9613960860256485,
  521919.0: 3.9613960860256485},
 'erode': {245384.0: 3.9613960860256485, 346468.0: 2.197387186244282},
 'unwittingly': {242243.0: 2.892994660705231, 465998.0: 3.107574760571518},
 'skydiving': {29024.0: 6.468932719880837,
  207521.0: 5.6022600707891685,
  263622.0: 3.9613960860256485,
  225991.0: 5.6022600707891685,
  73802.0: 7.922792172051297,
  431728.0: 7.4696800943855575,
  350545.0: 7.922792172051

In [ ]:
from collections import Counter

We will now use the original data to match the questions

In [ ]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


In [ ]:
#This function is used as the final call.
#In this function you can see that we are creating question_columns and then we pass into if condition
def original_data(qid_kind, que_kind):
  question_columns = ['question1', 'question2']
  length_of_column = len(question_columns)
  if que_kind == 'question':
    question_id_1 = df['qid1']
    questions = df.loc[(question_id_1 == qid_kind, question_columns[0])]
    question_loc_0 = questions.iloc[0]
    original_question = question_loc_0
  elif que_kind != 'question':
    question1 = df.loc[(df['qid2'] == qid_kind, question_columns[1])]
    question_loc_1_0 = question1.iloc[0]
    original_question = question_loc_1_0
  else:
    pass
  return original_question
def questions_columns():
  return columns_question[0]

Now we will create a final_result function that will help use differenctiate between the two Normalization techniques

In [ ]:
#this function is used to matching function for the Tf Idf
#it matches the matched_rsp and the function to find the top tf-idf function.
def final_result(norm_type, Rsp_cnt):
  rsp_tag = []
  matchingfunction = []
  for c in range(len(queries_100)):
    Matched_RSP = finaltfidf_top(norm_type, c, Rsp_cnt, show_matched=False)
    if queries_100.loc[c, 'qid2'] in Matched_RSP:
      rsp_tag[len(rsp_tag):] = [1.0]
    elif queries_100.loc[c, 'qid2'] not in Matched_RSP:
      rsp_tag[len(rsp_tag):] = [0.0]
    else:
      pass
  return rsp_tag

Now we will create a Function to summarize all the above functions as this function will keep count fo the questions and will compare and give the best results

In [ ]:
# Getting the top TF_IDF
#in this function we are keeping the count of the best matched questions and storing them for them to later be revealed
def TFIDF():
  return matchingfunction
def finaltfidf_top(norm_type, que_ID, rsp_count, show_matched):
    question = []
    item_cnt = []
    que_column = ['question1', 'qid1', 'qid2']
    length_of_question1 = len(question)
    question = queries_100.loc[que_ID, que_column[0]]
    ques_id_top = []
    qid_question = queries_100.loc[que_ID, que_column[1]]
    rsp_ID = queries_100.loc[que_ID, que_column[2]]
    finaltfidf = []
    if norm_type == 'l1':
        inv_typ = TF_IDF_L1
    elif norm_type == 'l2':
        inv_typ = TF_IDF_L2
    else:
        pass
    storing = []
    for word in question:
        if word in inv_typ.keys():
            i1 = inv_typ[word]
            ques_id_top
            item_count_len = len(item_cnt)
            item_cnt[item_count_len:] = [Counter(i1)]
            total = sum(item_cnt, Counter())
            tmp_rsp = sorted(total, key=total.get, reverse=True)
            rsp_matched = tmp_rsp[:rsp_count]
    storing_function =[]
    if show_matched == True:
        # Printing up the results
        print("Showing the Questions that are searched")
        print("Normalization method: {}".format(norm_type))
        function_to_store_count = 0
        typ_QA = ['question', 'result']
        print("\n"+original_data(qid_question, typ_QA[0]))
        matched_shows = []
        print("\nThe top {} matched questions:".format(rsp_count))
        for final_ID in rsp_matched:
            if final_ID == rsp_ID:
                print(original_data(final_ID, typ_QA[1]))
            elif final_ID != rsp_ID:
                print(original_data(final_ID, typ_QA[1]))
            else:
                pass
    elif show_matched == False:

        return rsp_matched
    else:
        pass

We will create a duplicate value to that only stores the column 'is_duplicate'

In [ ]:
top_duplicate = queries_100['is_duplicate']

Now lets find out the top 5 best questions that are a match to the question with ID = 18

In [ ]:
finaltfidf_top(norm_type='l1', que_ID=18, rsp_count=5, show_matched=True)

Showing the Questions that are searched
Normalization method: l1

How do I draw bending moment and shear force diagram for beams?

The top 5 matched questions:
What can I draw?
How do I draw bending moment and shear force diagram?
How do I draw the shear and moment diagram?
Why do you draw?
What is shear force?


to find the accuracy of the functions we will make use of the accuracy_score library

In [ ]:
#Now to get the accuracy of the top 5 from each norm

L1_norm_top5 = final_result('l1',5)
top5_of_L1 = accuracy_score(top_duplicate, L1_norm_top5)
print('Accuracy of the top 5 questions using Norm Type L1:', round(top5_of_L1 * 100), "%")

L2_norm_top5 = final_result('l1',5)
top5_of_L2 = accuracy_score(top_duplicate, L2_norm_top5)
print('Accuracy of the top 5 questions using Norm Type L2:', round(top5_of_L2 * 100), "%")

Accuracy of the top 5 questions using Norm Type L1: 27 %
Accuracy of the top 5 questions using Norm Type L2: 27 %


In [ ]:
#Now lets get the accuracy of the top 3 from each norm

L1_norm_top3 = final_result('l1',3)
top3_of_L1 = accuracy_score(top_duplicate, L1_norm_top3)
print('Accuracy of the top 3 questions using Norm Type L1:', round(top3_of_L1 * 100), "%")

L2_norm_top3 = final_result('l1',3)
top3_of_L2 = accuracy_score(top_duplicate, L2_norm_top3)
print('Accuracy of the top 3 questions using Norm Type L2:', round(top3_of_L2 * 100), "%")

Accuracy of the top 3 questions using Norm Type L1: 17 %
Accuracy of the top 3 questions using Norm Type L2: 17 %


#3. Glove Average word embedding and Glove Smooth Inverse Frequency

As there are already lots of changes being made to df1, we will use df2. We already did make a copy df2 of the original dataset df. Our first step here will be getting the dataset ready

####Text-Preprocessing

In [ ]:
df2

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0
...,...,...,...,...,...,...
363187,57443,100941,83372.0,How do I make money flying my drone?,How can I use a dji phantom to make money,1.0
363188,278260,62873,34460.0,What can you do with an economics degree?,What jobs can you get with an economics degree?,1.0
363189,136211,217377,217378.0,What type of current does a battery produce?,How does a generator work and produce current?,0.0
363190,302720,425744,285638.0,Grammar: What is difference between schedule a...,How do I understand the difference between the...,0.0


Checking for null values

In [ ]:
#checking if the dataset has any null values
print(df2.isnull().sum())

id               0
qid1             0
qid2             7
question1       11
question2       12
is_duplicate    12
dtype: int64


Dropping null values

In [ ]:
#dropping the null values from the dataset
df2.dropna(inplace = True)
print(df2.isnull().sum())

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64


In [ ]:
df2.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [ ]:
df2.shape

(363177, 6)

In [ ]:
df2.info()
df2.describe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


<bound method NDFrame.describe of             id    qid1      qid2  \
0       133273  213221  213222.0   
1       402555  536040  536041.0   
2       360472  364011  490273.0   
3       150662  155721    7256.0   
4       183004  279958  279959.0   
...        ...     ...       ...   
363187   57443  100941   83372.0   
363188  278260   62873   34460.0   
363189  136211  217377  217378.0   
363190  302720  425744  285638.0   
363191  126322   39774   20105.0   

                                                question1  \
0       How is the life of a math student? Could you d...   
1                     How do I control my horny emotions?   
2            What causes stool color to change to yellow?   
3                             What can one do after MBBS?   
4       Where can I find a power outlet for my laptop ...   
...                                                   ...   
363187               How do I make money flying my drone?   
363188          What can you do with an econo

In [ ]:
df2.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

Now we know that this dataset has 6 columns id, qid1, qid2, question1, question2, is_duplicate

In [ ]:
#lets print some of the questions, 1 from each column
df2['question1'][0]

'How is the life of a math student? Could you describe your own experiences?'

In [ ]:
df2['question2'][0]

'Which level of prepration is enough for the exam jlpt5?'

We can see that text preprocessing is required for the dataset

Applying lower casing function for question1

In [ ]:
#for column question 1
df2['question1'] = df2['question1'].apply(lower)

Applying lower casing function for question2

In [ ]:
#for column question 2
df2['question2'] = df2['question2'].apply(lower)

Checking for question1

In [ ]:
#checking
df2['question1'][7]

'what is the best self help book you have read? why? how did it change your life?'

checking for question2

In [ ]:
df2['question2'][7]

'what are the top self help books i should read?'

After lower casing we will remove html tags and special characters

Applying HTML tag removal function to question1

In [ ]:
#for column question 1
df2['question1'] = df2['question1'].apply(html_tags)

Applying HTML tag removal function to question2

In [ ]:
#for column question 2
df2['question2'] = df2['question2'].apply(html_tags)

Checking for question1

In [ ]:
#checking
df2['question1'][7]

'what is the best self help book you have read? why? how did it change your life?'

Checking for question2

In [ ]:
df2['question2'][7]

'what are the top self help books i should read?'

Now we will use the function we defined earler to remove all special characters

we will first start with the column 'question1'

In [ ]:
#for column question 1
df2['question1'] = df2['question1'].apply(special_character_removal)

we will now repeat the same thing for column 'question2'

In [ ]:
#for column question 2
df2['question2'] = df2['question2'].apply(special_character_removal)

Checking for question1

In [ ]:
#checking
df2['question1'][7]

'what is the best self help book you have read  why  how did it change your life '

Checking for question2

In [ ]:
df2['question2'][7]

'what are the top self help books i should read '

Now lets remove all the stop words

Appying stopwords removal for question1

In [ ]:
df2['question1'] = df2['question1'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

stopwords removal for question2

In [ ]:
df2['question2'] = df2['question2'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

Now lets check if all the stopwords have been removed or not

In [ ]:
#checking
df2['question1'][7]

'best self help book read change life'

In [ ]:
df2['question2'][7]

'top self help books read'

Tokenization and Lemmatization

applying tokenization function to df2 question1

In [ ]:
df2['question1'] = df2['question1'].apply(tokenization)

applying tokenization function to df2 question2

In [ ]:
df2['question2'] = df2['question2'].apply(tokenization)

We will now check for tokenization of the sentence for question1

In [ ]:
#checking
df2['question1'][7]

['best', 'self', 'help', 'book', 'read', 'change', 'life']

we will check for tokenization of sentence for question2

In [ ]:
df2['question2'][7]

['top', 'self', 'help', 'books', 'read']

We will now apply lemmatization on question1

In [ ]:
df2['question1'] = df2['question1'].apply(lemmatisation)

We will now apply lemmatizaton on question2

In [ ]:
df2['question2'] = df2['question2'].apply(lemmatisation)

Checking whether the lemmatization function is applied correctly or not

In [ ]:
#checking
df2['question1'][0]

['life', 'math', 'student', 'could', 'describe', 'experience']

In [ ]:
df2['question2'][7]

['top', 'self', 'help', 'book', 'read']

Reseting the Index so that it doesnot create indexing problems for us in future

In [ ]:
df2.reset_index(inplace=True)

Now we will remove all duplicates

In [ ]:
#We will now remove duplicates
list_of_data = ['qid2','question2']
rsp_final = df2[list_of_data]
questionid_2 = 'qid2'
rsp_final = rsp_final.drop_duplicates(questionid_2)
rsp_final = rsp_final.reset_index(drop=True)

we will now check whether the tables have made the changes that we meant for it

In [ ]:
df2.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describe, experie...","[level, prepration, enough, exam, jlpt5]",0.0
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
2,2,360472,364011,490273.0,"[cause, stool, color, change, yellow]","[cause, stool, come, little, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourne, airport]","[would, second, airport, sydney, australia, ne...",0.0


Now we will only take duplicate columns
i.e. when is_duplicate = 1.0

In [ ]:
#Taking the queries with duplicate = 1.0
#we will name them queries_2
queries_2 = df2[df2['is_duplicate'] != 0]

checking if the correct queries are selected

In [ ]:
queries_2

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
7,7,106969,147570,787.0,"[best, self, help, book, read, change, life]","[top, self, help, book, read]",1.0
11,11,233239,71243,177376.0,"[hillary, clinton, policy, towards, india, bec...","[hilary, clinton, policy, towards, india, beco...",1.0
13,13,11568,22332,22333.0,"[best, book, study, tensor, general, relativit...","[best, book, tensor, calculus]",1.0
...,...,...,...,...,...,...,...
363168,363183,300337,423121,423122.0,"[get, best, life]","[get, best, life]",1.0
363170,363185,344766,473025,473026.0,"[demand, oracle, atg, market]","[demand, oracle, atg, market]",1.0
363172,363187,57443,100941,83372.0,"[make, money, flying, drone]","[use, dji, phantom, make, money]",1.0
363173,363188,278260,62873,34460.0,"[economics, degree]","[job, get, economics, degree]",1.0


Now we will take the first 100 queries for searching the best matched questions

In [ ]:
#Taking the first 100 queries for the search matching
queries_2_100 = queries_2[:100]

Checking the operation we just performed

In [ ]:
queries_2_100.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
1,1,402555,536040,536041.0,"[control, horny, emotion]","[control, horniness]",1.0
3,3,150662,155721,7256.0,"[one, mbbs]",[mbbs],1.0
7,7,106969,147570,787.0,"[best, self, help, book, read, change, life]","[top, self, help, book, read]",1.0
11,11,233239,71243,177376.0,"[hillary, clinton, policy, towards, india, bec...","[hilary, clinton, policy, towards, india, beco...",1.0
13,13,11568,22332,22333.0,"[best, book, study, tensor, general, relativit...","[best, book, tensor, calculus]",1.0


Rearranging the columns for better conventions

In [ ]:
#lets rearrange the columns so that qid1 is next to question1
column2 = ["qid1","question1","qid2","question2","is_duplicate"]
queries_2_100 = queries_2_100[column2]


Again we will reset index to make sure that there is no indexing error in future

In [ ]:
queries_2_100 = queries_2_100.reset_index(drop=True)

checking whether the index now starts from 0

In [ ]:
queries_2_100.head()

,qid1,question1,qid2,question2,is_duplicate
0,536040,"[control, horny, emotion]",536041.0,"[control, horniness]",1.0
1,155721,"[one, mbbs]",7256.0,[mbbs],1.0
2,147570,"[best, self, help, book, read, change, life]",787.0,"[top, self, help, book, read]",1.0
3,71243,"[hillary, clinton, policy, towards, india, bec...",177376.0,"[hilary, clinton, policy, towards, india, beco...",1.0
4,22332,"[best, book, study, tensor, general, relativit...",22333.0,"[best, book, tensor, calculus]",1.0


####3.1 GloVe Average Word Embedding

We will directly use the pretrained library for this as provided in the instruction section of the assignment.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-05-28 01:39:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-28 01:39:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-28 01:39:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#unzipping the library
!unzip glove*zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


GloVe stands for global vectors for word representation. It is an unsupervised learning algorithm developed by Stanford for generating word embeddings by aggregating global word-word co-occurrence matrix from a corpus.

In [ ]:
glove = dict()
with open('/content/glove.6B.300d.txt','r') as embedding:
  for items in embedding:
    splitting_item = items.split()
    splitvalue = splitting_item
    floatvalue = "float32"
    vector = np.asarray(splitvalue[1:],floatvalue)
    glove[splitvalue[0]] = vector

Cosine similarity is the cosine of the angle between two vectors and it is used as a distance evaluation metric between two points in the plane. The cosine similarity measure operates entirely on the cosine principles where with the increase in distance the similarity of data points reduces.


We define cosine similarity mathematically as the dot product of the vectors divided by their magnitude. For example, if we have two vectors, A and B, the similarity between them is calculated as:
cosinesimilarity = A.B/ ||A||||B||

In [ ]:
#now we will define a function to calculate cosine similarity
def similarity_using_cosine(vector1, vector2):
  return vector1.vector2 / vector1 * vector2
def cosine_similarity(vector1, vector2):
  number2 = np.dot(vector1, vector2)
  vector1_main = np.linalg.norm(vector1, ord=2)
  vector2_main = np.linalg.norm(vector2, ord=2)
  multiplication_of_vector = vector1_main * vector2_main
  #Multiplication of vectors
  multivector12 = multiplication_of_vector
  #division of vectors
  division = number2 / multivector12
  #cosinesimilarity
  cosinesimilarity = division
  return cosinesimilarity

we will prepare the original data function so that the we can compare the results

In [ ]:
def original_data(qid_kind, que_kind):
  question_columns = ['question1', 'question2']
  if que_kind == 'question':
    question_id_1 = df['qid1']
    questions = df.loc[(question_id_1 == qid_kind, question_columns[0])]
    question_loc_0 = questions.iloc[0]
    original_question = question_loc_0
    length_of_question2 = len(question_id_1)
  elif que_kind != 'question':
    question1 = df.loc[(df['qid2'] == qid_kind, question_columns[1])]
    question_loc_1_0 = question1.iloc[0]
    original_question = question_loc_1_0
  else:
    pass
  return original_question

Now we will create a function to generate the result using the GloVe result

In [ ]:
def gloveresult(que_ID, matrix_rsp, que_matrix, rsp_count, show_matched):
  itemmatching = []
  rspmatching = []
  qid_of_question_2 = queries_2_100.loc[que_ID, 'qid1']
  rspID2 = queries_2_100.loc[que_ID, 'qid2']
  storing_glove_result = []
  for t in range(len(matrix_rsp)):
    vector1 = que_matrix[que_ID, :]
    vector2 = matrix_rsp[t, :]
    length_of_item = len(itemmatching)
    itemmatching[length_of_item:] = [cosine_similarity(vector1, vector2)]
  storing_itemmatching = []
  itemmatching = np.array(itemmatching)
  nan9999 = nan=-9999
  length_of_itemmetaching = len(storing_glove_result)
  itemmatching = np.nan_to_num(itemmatching, nan9999)
  sorting = itemmatching.argsort()
  sorting = sorting[::-1][:rsp_count]
  storing_sorting = []
  for rsp_id in sorting:
    responseqid2 = rsp_final.loc[rsp_id, 'qid2']
    length_of_rsp = len(rspmatching)
    rspmatching[length_of_rsp:] = [responseqid2]
  storing_rsp_id = []
  if show_matched == True:
    print("Showing the questions that are searched")
    print("\n"+original_data(qid_of_question_2,'question'))
    print("\n The best {} matched questions are:".format(rsp_count))
    for response in rspmatching:
      print(original_data(response,'result'))
  elif show_matched == False:
    return rspmatching
  else:
    pass

Now we will define a Class for calculating the average word vectors and comparision of the two datasets

In [ ]:
#Now lets define a class for calculating the average word vectors
class Avg_Word_Vector():
  def __init__(self):
    pass
  avg_word_vector = []
  def fitting(self, Y):
    return self
  def convert_glove(self, X):
    return self
  X = []
  length_glove = len(X)
  def convertion(self, embedding, item, hb):
    self.embedding = embedding
    self.item = item
    aa = self.embedding[hb]
    self.dm = aa.shape[0]
    #Matric of zeros
    Z = np.zeros(shape = (len(item), self.dm))
    #For Average Words
    storing_the_glove_vector = []
    for g in range(len(item)):
      words = item[g]
      vector_2 = list()
      length_glove_vector = len(storing_the_glove_vector)
      for word in words:
        try:
          aa = self.embedding[word]
          length_of_vector = len(vector_2)
          vector_2[length_of_vector:] = [aa]
        except KeyError:
          pass
        except:
          print("Error")
      vector_2 = np.array(vector_2)
      length_of_vector = len(vector_2)
      if 0 < len(vector_2):
        temp_embedding = np.mean(vector_2, axis=0)
        fin_embedding = temp_embedding.reshape(1,-1)
        Z[g,:] = fin_embedding
      else:
        pass
    return Z

Now we will use the class that we have just created and use the convertion function definied in the class

In [ ]:
class_of_word_vector = Avg_Word_Vector()
mean_of_responses = class_of_word_vector.convertion(glove, rsp_final['question2'],'word')
mean_of_queries = class_of_word_vector.convertion(glove, queries_2_100['question1'],'word')

Now we will generate the top 5 best searched results for the specified question number

For the result given below we are generating the top 5 most similar questions to question number 19

In [ ]:
gloveresult(19, mean_of_responses, mean_of_queries, 5, True)

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Showing the questions that are searched

What's the easiest way to learn Java programs?

 The best 5 matched questions are:
What is the easiest way to learn java programming?
How can you learn Java easy way?
What is the best way to learn Java?
What is the best way to learn Java programming?
What is a fun way to learn Java?


Similarly we will also find the top 3 most similar questions to question number 19

In [ ]:
#Now for the top 3
gloveresult(19, mean_of_responses, mean_of_queries, 3, True)

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Showing the questions that are searched

What's the easiest way to learn Java programs?

 The best 3 matched questions are:
What is the easiest way to learn java programming?
How can you learn Java easy way?
What is the best way to learn Java?


Now we will create a list to store the accuracy of the top 3 and top 5 results generated

In [ ]:
#creating a list to store accuracy of top 3 and top 5
accuracy_of_top3 = list()
accuracy_of_top5 = list()

Now we will write a formula to calculate the accuracy

In [ ]:
#Now we calculate the accuracy
for index_of_query in range(len(queries_2_100)):
  question_id2 = 'qid2'
  question_id_1 = 'qid1'
  top_duplicate = queries_2_100.loc[index_of_query, question_id2]
  ranked_top5 = gloveresult(index_of_query, mean_of_responses, mean_of_queries, 5, False)
  length_of_top3 = len(accuracy_of_top3)
  length_of_top5 = len(accuracy_of_top5)
  length_of_question = len(question_id_1)
  # IF function to generate the top3 of the best matched questions
  if top_duplicate in ranked_top5[:3]:
    accuracy_of_top3[length_of_top3:] = [1.0]
    accuracy_of_top5[length_of_top5:] = [1.0]
  # elif we will take the top 5
  elif top_duplicate in ranked_top5:
    accuracy_of_top3[length_of_top3:] = [0.0]
    accuracy_of_top5[length_of_top5:] = [1.0]
  #else we will not get any accuracy
  else:
    accuracy_of_top3[length_of_top3:] = [0.0]
    accuracy_of_top5[length_of_top5:] = [0.0]

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Calculating the value of accuracy using accuracy_score and printing the final accuracy

In [ ]:
final_accuracy_of_top3 = accuracy_score(queries_2_100['is_duplicate'], accuracy_of_top3)
final_accuracy_of_top5 = accuracy_score(queries_2_100['is_duplicate'], accuracy_of_top5)
print("Accuracy of the top 3 matching question:", round(final_accuracy_of_top3 * 100), "%")
print("Accuracy of the top 5 matching question:", round(final_accuracy_of_top5 * 100), "%")

Accuracy of the top 3 matching question: 50 %
Accuracy of the top 5 matching question: 58 %


####3.2 GloVe Smooth Inverse Frequency

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from sklearn.decomposition import TruncatedSVD

Smooth Inverse Frequency, a weighting scheme to improve performance of sentence embeddings. When encoding a sentence, it is important to identify which words in the sentence are more significant.

For example, if calculating the embedding of the sentence “who was Mozart?” the word “was” doesn’t add much meaning; looking for sentences or documents relating to the word “was” will not yield any useful results for the original question. It’s clear that “Mozart” holds the most meaning in the question from a human standpoint, but how do you program a machine to identify that? SIF operates under the assumption that the most important words tend to also be used less frequently.

Now we will define a function to use Smooth Inverse Frequency

In [ ]:
#Smooth Inverse Frequency
def weight_of_SIF(SIF_corpus, alpha=1e-3):
  lambda_x = (lambda x: x)
  CountV = CountVectorizer(tokenizer=None, analyzer = lambda_x, preprocessor=None, lowercase=False)
  Hb = CountV.fit_transform(SIF_corpus)
  wsif = []
  #calculating probability
  xa = 0
  frequencys = np.sum(Hb, axis = xa)
  dividesif = (frequencys/np.sum(frequencys))
  probabilitys = dividesif
  smoothinversefrequency_value = []
  #creting weighted words
  coefficientsif = alpha/(alpha + probabilitys)
  wordsif = defaultdict(lambda: coefficientsif.min())
  wordsif = {wd:coefficientsif[xa,i] for wd,i in CountV.vocabulary_.items()}
  return wordsif
word_list = list(rsp_final['question2'])
finalsif_weight = weight_of_SIF(word_list, alpha=1e-3)
def SIF_weight_1(self, embedding, item, th):
  self.embedding = embedding
  self.item = item
  Sif_weight_1 = []
  return Sif_weight_1
def SIF_weight_2(dt, embedding, wd_weights):
  word_sif = 'word'
  sif = embedding[word_sif]
  wsif = sif.shape[0]
  W = np.zeros(shape = (len(dt), wsif))
  sif_weight_2 = []
  for j in range(len(dt)):
    sdw = dt[j]
    vector_3 = list()
    vector_4 = list()
    for word in sdw:
      try:
        tempo_embedding = embedding[word]
        tempoweight = wd_weights[word]
        vector_3[len(vector_3):] = [(tempo_embedding * tempoweight)]
      except KeyError:
        pass
      except:
        print("Error")
    vector_3 = np.array(vector_3)
    length_of_vector3 = len(vector_3)
    length_of_vector4 = len(vector_4)
    if 0 < length_of_vector3:
      temporary_sum = np.sum(vector_3, axis=0)
      final_sum = temporary_sum.reshape(1,-1)
      W[j,:] = final_sum / length_of_vector3
      K = length_of_vector4
  return W
respective_weight = SIF_weight_2(rsp_final['question2'], glove, finalsif_weight)
query_downweight = SIF_weight_2(queries_2_100['question1'], glove, finalsif_weight)


def weightcalculation(sif_weight_2,Sif_weight_1):
  return sif_weight_2
def pccalculation(X_train):
  computer_one = 1
  truncsvd = TruncatedSVD(n_components = computer_one, n_iter=7, random_state=0)
  truncsvd.fit(X_train[:10000, :])
  componentprint = truncsvd.components_
  return componentprint

pc_number_1 = pccalculation(respective_weight)
def sifpc(vector_3, vector_4):
  return vector_3
def pcremoval(W, componentprint):
  WW = W - W.dot(componentprint.transpose()) * componentprint
  return WW
def sifpc2(vector_3, vector_4):
  return vector_4
RespectiveSIF = pcremoval(respective_weight, pc_number_1)
QueueSIF = pcremoval(query_downweight, pc_number_1)

Now we will generate the top 5 best matched questions for the specfied question using Smooth Inverse Frequency

For this specific result we will generate the top 5 most searched result for the question number 19

In [ ]:
gloveresult(19, RespectiveSIF, QueueSIF, 5, True)

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Showing the questions that are searched

What's the easiest way to learn Java programs?

 The best 5 matched questions are:
What is the easiest way to learn java programming?
What exactly is Java used for? What things can I program using Java?
What is the easiest way to write a program?
How can you learn Java easy way?
Is Java easy to learn?


we will also generate the top 3 most searched question for the question number 19

In [ ]:
gloveresult(19, RespectiveSIF, QueueSIF, 3, True)

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Showing the questions that are searched

What's the easiest way to learn Java programs?

 The best 3 matched questions are:
What is the easiest way to learn java programming?
What exactly is Java used for? What things can I program using Java?
What is the easiest way to write a program?


Now we will create a list to store the accuracy of the top 3 and top 5 results generated

In [ ]:
#creating a list to store accuracy of top 3 and top 5
SIFaccuracy_of_top3 = list()
SIFaccuracy_of_top5 = list()

Now we will write a formula to calculate the accuracy

In [ ]:
#Calculating the accuracy of the GloVe Smooth Inverse Frequency
for index_of_query in range(len(queries_2_100)):
  list_of_qid = 'qid2'
  idrsp3 = queries_2_100.loc[index_of_query, list_of_qid]
  list_of_qid_2 = 'qid1'
  SIFtop5 = gloveresult(index_of_query, RespectiveSIF, QueueSIF, 5, False)
  length_of_SIFacctop3 = len(SIFaccuracy_of_top3)
  length_of_SIFacctop5 = len(SIFaccuracy_of_top5)
  length_of_qid = len(list_of_qid_2)
  #if function to create the take the top 3 from the top 5 for accuracy
  if idrsp3 in SIFtop5[:3]:
    SIFaccuracy_of_top3[length_of_SIFacctop3:] = [1.0]
    SIFaccuracy_of_top5[length_of_SIFacctop5:] = [1.0]
  #elif function to take the top 5 for accuracy
  elif idrsp3 in SIFtop5:
    SIFaccuracy_of_top3[length_of_SIFacctop3:] = [0.0]
    SIFaccuracy_of_top5[length_of_SIFacctop5:] = [1.0]
  #else we dont get accuracy
  else:
    SIFaccuracy_of_top3[length_of_SIFacctop3:] = [0.0]
    SIFaccuracy_of_top5[length_of_SIFacctop5:] = [0.0]

<ipython-input-129-cb26da971252>:12: RuntimeWarning: invalid value encountered in double_scalars
  division = number2 / multivector12


Calculating the value of accuracy using accuracy_score and printing the final accuracy

In [ ]:
final_SIFaccuracy_of_top3 = accuracy_score(queries_2_100['is_duplicate'], SIFaccuracy_of_top3)
final_SIFaccuracy_of_top5 = accuracy_score(queries_2_100['is_duplicate'], SIFaccuracy_of_top5)

print("Accuracy of the top 3 question when Smooth Inverse Frequency GloVe is used is:", round(final_SIFaccuracy_of_top3 * 100), "%")
print("Accuracy of the top 5 question when Smooth Inverse Frequency GloVe is used is:", round(final_SIFaccuracy_of_top5 * 100), "%")

Accuracy of the top 3 question when Smooth Inverse Frequency GloVe is used is: 44 %
Accuracy of the top 5 question when Smooth Inverse Frequency GloVe is used is: 51 %
